In [1]:
from datasets import load_dataset
from random import randrange
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer
import mlflow
import shutil
from os.path import dirname

In [2]:
def remove_dir(dir_path):
    try:
        shutil.rmtree(dir_path)
        print(f"Folder '{dir_path}' has been deleted.")
    except Exception as e:
        # Ignore errors, you can print a message if needed
        print(f"Folder '{dir_path}' has been deleted.")

#base_model = "codegen-350M-multi" #Remember to apply this value in the argument parameter of tokenizer, to use local folder instead of attempting to download from HF
base_model = "codegen-350M-multi"
base_model_name = "codegen-350M-multi"
merged_model = "merged_ft_model"
training_output = "training_output" # stores the checkpoints
#dataset_name = "Shreyasrp/Text-to-SQL"
dataset_name = "text-to-sql_dataset"
split = "train[:10%]"
device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Device to be used is {device_map}")
print(f"--------------------------------------\n")
# Remove the model folders if exist
remove_dir(training_output) 
remove_dir(merged_model)

Device to be used is cuda:0
--------------------------------------

Folder 'training_output' has been deleted.
Folder 'merged_ft_model' has been deleted.


In [3]:
# BitsAndBytesConfig config
#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_compute_dtype="float16"
#)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

peft_config = LoraConfig(
      r=16,
      lora_alpha=32,
      lora_dropout=0.05,
      bias="none",
      task_type="CAUSAL_LM",
)

def prompt_instruction_format(sample):
  return f"""Context:
    {sample['instruction']}

    Result:
    {sample['output']}
    """

In [4]:
dataset = load_dataset(dataset_name, split=split)

In [5]:
#base_model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, use_cache = False, device_map=device_map)
base_model = AutoModelForCausalLM.from_pretrained(base_model, use_cache = False, device_map=device_map)

In [6]:
def print_param_precision(model):
  dtypes = {}
  for _, p in model.named_parameters():
      dtype = p.dtype
      if dtype not in dtypes:
          dtypes[dtype] = 0
      dtypes[dtype] += p.numel()
  total = 0
  for k, v in dtypes.items():
      total += v
  for k, v in dtypes.items():
      print(f"{k}, {v / 10**6:.4f} M, {v / total*100:.2f} %")

def print_trainable_parameters(model):
  # Count the total parameters
  total_params = sum(p.numel() for p in model.parameters())
  print(f"Total parameters: {total_params/10**6:.4f} M")

  # Count the trainable parameters
  trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print(f"Trainable parameters: {trainable_params/10**6:.4f} M")
    

In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"{device} Memory Used: {base_model.get_memory_footprint() / 1024**2:.4f} MB")
print(f"--------------------------------------\n")
print(f"Parameters loaded for model {base_model_name}:")
print_trainable_parameters(base_model)
print("\n")
print(f"Data types for loaded model {base_model_name}:")
print_param_precision(base_model)

#for name, param in base_model.named_parameters():
#    print(f"Parameter name: {name}, Data type: {param.dtype}")

cuda:0 Memory Used: 1440.7500 MB
--------------------------------------

Parameters loaded for model codegen-350M-multi:
Total parameters: 356.7124 M
Trainable parameters: 356.7124 M


Data types for loaded model codegen-350M-multi:
torch.float32, 356.7124 M, 100.00 %


In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.deprecation_warnings["Asking-to-pad-a-fast-tokenizer"] = True

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
trainingArgs = TrainingArguments(
    output_dir=training_output,
    num_train_epochs=3,
    #per_device_train_batch_size=4,
    auto_find_batch_size=True,
    #gradient_checkpointing=True, # When enabled, memory can be freed at the cost of small decrease in the training speed due to recomputing parts of the graph during back-propagation.
    #gradient_accumulation_steps=2,
    #optim="paged_adamw_32bit",
    #optim="paged_adamw_8bit",
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    #fp16=False,
    #bf16=False,
    #lr_scheduler_type="cosine",
    disable_tqdm=True
)

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [13]:
print("Start Fine-Tuning")
mlflow.set_experiment("Fine-tune TRL")
trainer.train()
print("Training Done")

Start Fine-Tuning
{'loss': 1.4677, 'learning_rate': 0.0001996609020006782, 'epoch': 0.01}
{'loss': 1.2769, 'learning_rate': 0.0001993218040013564, 'epoch': 0.01}
{'loss': 1.0401, 'learning_rate': 0.00019898270600203458, 'epoch': 0.02}
{'loss': 1.0126, 'learning_rate': 0.0001986436080027128, 'epoch': 0.02}
{'loss': 0.9877, 'learning_rate': 0.00019830451000339098, 'epoch': 0.03}
{'loss': 0.9136, 'learning_rate': 0.0001979654120040692, 'epoch': 0.03}
{'loss': 0.8055, 'learning_rate': 0.00019762631400474737, 'epoch': 0.04}
{'loss': 0.8521, 'learning_rate': 0.00019728721600542558, 'epoch': 0.04}


/home/cdsw/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.7954, 'learning_rate': 0.0001969481180061038, 'epoch': 1.0}
{'loss': 0.8338, 'learning_rate': 0.00019660902000678197, 'epoch': 1.01}
{'loss': 0.829, 'learning_rate': 0.00019626992200746018, 'epoch': 1.01}
{'loss': 0.819, 'learning_rate': 0.00019593082400813836, 'epoch': 1.02}
{'loss': 0.7047, 'learning_rate': 0.00019559172600881657, 'epoch': 1.02}
{'loss': 0.8098, 'learning_rate': 0.00019525262800949476, 'epoch': 1.03}
{'loss': 0.7965, 'learning_rate': 0.00019491353001017294, 'epoch': 1.03}
{'loss': 0.7758, 'learning_rate': 0.00019457443201085115, 'epoch': 1.04}


/home/cdsw/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.8183, 'learning_rate': 0.00019423533401152933, 'epoch': 2.0}
{'loss': 0.7617, 'learning_rate': 0.00019389623601220754, 'epoch': 2.01}
{'loss': 0.7363, 'learning_rate': 0.00019355713801288572, 'epoch': 2.01}
{'loss': 0.7448, 'learning_rate': 0.00019321804001356393, 'epoch': 2.02}
{'loss': 0.7363, 'learning_rate': 0.00019287894201424212, 'epoch': 2.02}
{'loss': 0.7029, 'learning_rate': 0.0001925398440149203, 'epoch': 2.03}
{'loss': 0.7377, 'learning_rate': 0.0001922007460155985, 'epoch': 2.03}
{'loss': 0.7109, 'learning_rate': 0.00019186164801627672, 'epoch': 2.04}
{'train_runtime': 502.6616, 'train_samples_per_second': 46.898, 'train_steps_per_second': 5.867, 'train_loss': 0.8594541530298992, 'epoch': 2.04}
Training Done


In [14]:
# save model
trainer.save_model() # adapter models
print("Model Saved")

Model Saved


In [15]:
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    trainingArgs.output_dir,
    return_dict=True,
    device_map=device_map
)

# Merge LoRA adapter with the base model and save the merged model
lora_merged_model = trained_model.merge_and_unload()
lora_merged_model.save_pretrained(merged_model)
tokenizer.save_pretrained(merged_model)

('merged_ft_model/tokenizer_config.json',
 'merged_ft_model/special_tokens_map.json',
 'merged_ft_model/vocab.json',
 'merged_ft_model/merges.txt',
 'merged_ft_model/added_tokens.json',
 'merged_ft_model/tokenizer.json')

In [ ]:
merged_model = "merged_ft_model"
merged_model = "quantized_model1"
device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
model_new = AutoModelForCausalLM.from_pretrained(merged_model, device_map="auto")

print(f"{device_map} Memory Used: {model_new.get_memory_footprint() / 1024**2:.4f} MB")
print("\nParameters:")
print_trainable_parameters(model_new)
print("\nData types:")
print_param_precision(model_new)

In [ ]:
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    trainingArgs.output_dir,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

# Merge LoRA adapter with the base model and save the merged model
lora_merged_model = trained_model.merge_and_unload()
lora_merged_model.save_pretrained(merged_model,safe_serialization=True)
tokenizer.save_pretrained(merged_model)

In [ ]:
# Reset the iPython kernel before proceeding with running the following test.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device_map)

#loaded_models = {}
mymodel1 = "quantized_model1"
ft_tokenizer = AutoTokenizer.from_pretrained(mymodel1,torch_dtype=torch.bfloat16,device_map=device_map)
ft_model = AutoModelForCausalLM.from_pretrained(mymodel1,torch_dtype=torch.bfloat16,device_map=device_map)
#loaded_models[mymodel1] = {"tokenizer": tokenizer1, "model": model1}
#tokenizer1 = loaded_models[mymodel1]["tokenizer"]
#base_model = loaded_models[mymodel1]["model"]

#loaded_models = {}
base_model = "codegen-350M-multi"
base_tokenizer = AutoTokenizer.from_pretrained(base_model,torch_dtype=torch.bfloat16,device_map=device_map)
base_model = AutoModelForCausalLM.from_pretrained(base_model,torch_dtype=torch.bfloat16,device_map=device_map)
#loaded_models[mymodel2] = {"tokenizer": tokenizer, "model": model}
#tokenizer2 = loaded_models[mymodel2]["tokenizer"]
#lora_merged_model = loaded_models[mymodel2]["model"]

In [ ]:
mytask="CREATE TABLE trip (bus_stop VARCHAR, duration INTEGER), list all the bus stops from which a trip of duration below 100 started."
prompt = f"""
# Instruction:
Use the context below to produce the result
# context:
{mytask}
# result:
"""

input_id1 = ft_tokenizer.encode(prompt, return_tensors="pt").to(device_map)
input_id2 = base_tokenizer.encode(prompt, return_tensors="pt").to(device_map)
attention_mask1 = torch.ones(input_id1.shape, dtype=torch.long).to(device_map)
attention_mask2 = torch.ones(input_id2.shape, dtype=torch.long).to(device_map)
print(f"--------------------------------------\n")
print(f"Prompt:\n{prompt}\n")
print(f"--------------------------------------\n")

print(f"Fine-tuned Model Result :\n")
output_base = ft_model.generate(input_ids=input_id1, do_sample=True, max_new_tokens=100, top_p=0.9,temperature=0.5,attention_mask=attention_mask1)
print(f"{ft_tokenizer.batch_decode(output_base.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"--------------------------------------\n")

print(f"Base Model Result :\n")
output_ft = base_model.generate(input_ids=input_id2, do_sample=True, max_new_tokens=100, top_p=0.9,temperature=0.5,attention_mask=attention_mask2)
print(f"{base_tokenizer.batch_decode(output_ft.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")